<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/ON5_GamblersRuin_v2_For_Stock_Exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Two Absorbing States: Gambler's Ruin**

Now, consider the following situation. A gambler bets on the outcome of a sequence of independent fair coin tosses. With each heads, the gambler gains one dollar. With each tails, the gambler loses one dollar. The gambler stops betting after reaching a fortune of $\overline{S}$ dollars or after emptying their pockets.

*   What are the probabilities of each stopping outcome?
*   How long will it take for the gambler, in expectation, to arrive at one of the stopping outcomes?

To answer these questions, we can model this setting as a Markov chain on the state space $\mathcal{S}\in\{0,1,...,\overline{s}\}$. The gambler starts with initial money $k\in\mathcal{S}$, and $s_t$ represents the money in the gambler's pocket at time $t$. Thus, we have that, for $0\lt s_t \lt \overline{s}$:

*   $\mathbb{P}(s_{t+1}=s_t+1|s_{t})=0.5$
*   $\mathbb{P}(s_{t+1}=s_t-1|s_{t})=0.5$

States 0 and $\overline{s}$ are absorbing states because any sequence of draws from the Markov chain stops after reaching any of those situations. Alternatively, we can think that $\mathbb{P}(s_{t+1}=s_t|s_{t}=\overline{s})=\mathbb{P}(s_{t+1}=s_t|s_{t}=0)=1$. We can then represent the $(\overline{s}+1)\times(\overline{s}+1)$ transition matrix as:
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 &\cdots & 0 \\
0.5 & 0 & 0.5 & 0 & 0 &\cdots & 0 \\
0 & 0.5 & 0 & 0.5 & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots& \cdots & \vdots \\
0 & 0 & 0 & 0.5 & 0 & 0.5 & 0 \\
0 & 0 & 0 & 0 & 0.5 & 0 & 0.5 \\
0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\end{align}
$$
Before solving this with math, let's see some Monte Carlo simulation results (in this example, the gambler stops betting after reaching a fortune of 5 USD and starts with 1 USD):

In [80]:
# below code modified for Stock Exchange
# https://github.com/raklokesh/GamblersRuin_RandomWalks/blob/master/GamblersRuin.py


# Random walk of gamblers money to show Gambler Ruin in roulette
# Gambler tries to achieve a goal money by betting one dollar every spin in roulette
# What is the probability that the gambler is not ruined i.e. gambler doesnt run out of money?
# Run simulations of several episodes and count proportion of episodes in which the gambler is not ruined! Each episode ends when the gambler is ruined or when he achieves goal money.


# PWIN = 0.77, for %100, mean wait = 206 days
# PWIN = 0.759, for %100, mean wait = 215 days ENKAI


# PWIN = 1,    for %5, mean wait = 6 days
# PWIN = 0.9,  for %5, mean wait = 6 days
# PWIN = 0.85, for %5, mean wait = 7 days
# PWIN = 0.8,  for %5, mean wait = 8.4 days
# PWIN = 0.75, for %5, mean wait = 10 days
# PWIN = 0.7,  for %5, mean wait = 12 days
# PWIN = 0.65, for %5, mean wait = 17 days
# PWIN = 0.6,  for %5, mean wait = 27 days
# PWIN = 0.55, for %5, mean wait = 56 days
# PWIN = 0.51, for %5, mean wait = 364 days

PWIN = 0.759
TARGET_PURSE = 200
INIT_PURSE = 100
MEAN_DAILY_RETURN = 0.00625


import numpy as np

class Gambler():
    def __init__(self):
        self.initial_money=INIT_PURSE
        self.current_money=self.initial_money
        self.goal_money=TARGET_PURSE
        self.bets_won=0
        self.trials=0

    def update_money(self,outcome):
        self.current_money+=outcome
        if outcome>0:
            self.bets_won+=1
            self.trials+=1
        else:
            self.trials+=1

        if self.current_money>=self.goal_money or self.current_money<=0:
            result=self.current_money/self.goal_money
            return True,result,self.trials,self.current_money
        else:
            return False,0,self.trials,self.current_money


class Roulette():
    def __init__(self):
        self.win_prob= PWIN

    def play_roulette(self,current_money):
        fate=np.random.random_sample()
        if fate < self.win_prob:
            outcome=current_money*MEAN_DAILY_RETURN
        else:
            outcome=-current_money*MEAN_DAILY_RETURN

        return outcome


episodes=10000
finished_episodes=np.arange(int(episodes/10),episodes,int(episodes/10))

gambler = Gambler()
roulette = Roulette()

episode_trials=np.zeros(episodes)
episode_wins=0
episode_spins=np.zeros(episodes)
episode_betswon=np.zeros(episodes)
current_money = INIT_PURSE
for i in range(episodes):
    if i in finished_episodes:
        print('Completed {} episodes'.format(i))

    end = False
    gambler.__init__()
    spins=0
    TR = 0
    while end==False:
        outcome=roulette.play_roulette(current_money)
        end,result,trials, current_money = gambler.update_money(outcome)
        spins+=1
    episode_trials[i]=trials
    episode_wins+=result
    episode_spins[i]=spins
    episode_betswon[i]=gambler.bets_won


print('The probability of winning from simulation of {} episodes is {} in {} days, if daily return is %{}'.format(episodes,episode_wins/episodes,trials, 100*MEAN_DAILY_RETURN))
print('The average number of roulette spins per episode is: ', np.mean(episode_spins))
print('The average number of trials per episode is: ', np.mean(episode_trials))


# Reference : http://web.mit.edu/neboat/Public/6.042/randomwalks.pdf
# Using recursive relation we can obtain true probability of achieving the goal amount using the formula shown below
p=roulette.win_prob

T=gambler.goal_money
n=gambler.initial_money
true_probability=(((1-p)/p)**n-1)/(((1-p)/p)**T-1)
print("The true probability of winning is {}".format(true_probability))



Completed 1000 episodes
Completed 2000 episodes
Completed 3000 episodes
Completed 4000 episodes
Completed 5000 episodes
Completed 6000 episodes
Completed 7000 episodes
Completed 8000 episodes
Completed 9000 episodes
The probability of winning from simulation of 10000 episodes is 1.0026204010419748 in 231 days, if daily return is %0.625
The average number of roulette spins per episode is:  215.2117
The average number of trials per episode is:  215.2117
The true probability of winning is 1.0
